In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
from collections import Counter
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
import numpy as np 
import pandas as  pd
import json
import random
import re
import requests
from tqdm import tqdm
from faker import Faker
random.seed(10)

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1020)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1020)>


False

## Assigning Reviewer to a research paper
1. Extract the authors for each domain
2. For each paper remove the authors who worked on the paper
3. Now form the panel of authors with high h index 
4. Randomly allocate 2 or 3 reviewers for each paper

In [ ]:
import json
import os

# Directory containing JSON files
json_dir = '/Users/elnararb/Documents/UPC/Semantic Data Management/data/paper_details'

# List to hold combined data
combined_papers_data = []
seen_ids = set()

# Loop through each JSON file in the directory
for filename in os.listdir(json_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(json_dir, filename)
        with open(file_path, 'r') as file:
            data = json.load(file)  # Load JSON data
            # Filter data
            filtered_data = [
                paper for paper in data
                if paper is not None and paper.get("publicationTypes") is not None and ("JournalArticle" in paper.get("publicationTypes", [])
                    or "Conference" in paper.get("publicationTypes", []))
            ]
            for paper in filtered_data:
                paper_id = paper.get("paperId")
                journal = paper.get("journal", [])
                if "JournalArticle" in paper.get("publicationTypes", []) and journal is None and "Conference" not in paper.get("publicationTypes", []):
                    continue
                if "JournalArticle" in paper.get("publicationTypes", []) and "Conference" not in paper.get("publicationTypes", []) and (journal.get("volume") is None or journal.get("volume")==""):
                    continue
                if paper_id and paper_id not in seen_ids:
                    seen_ids.add(paper_id)
                    combined_papers_data.append(paper)

# Write combined data to a new JSON file
with open('/Users/elnararb/Documents/UPC/Semantic Data Management/data/combined_papers_data.json', 'w') as outfile:
    json.dump(combined_papers_data, outfile, indent=4)

print("JSON files combined successfully!")

In [4]:
## Load Papers
import json
paper_lst = []
paths = ['combined_papers_data.json']
for path in paths:
    with open(f'/Users/elnararb/Documents/UPC/Semantic Data Management/data/{path}','r') as f:
        data = json.load(f)

    paper_lst.extend(data)

In [64]:
## Create the domain of the authors 
domain_author = {}

for paper in paper_lst:
    if paper['fieldsOfStudy'] != None:
        for domain in paper['fieldsOfStudy']:
            for author in paper['authors']:
                if domain not in domain_author.keys():
                    domain_author[domain] = [author['authorId']]
                else:
                    domain_author[domain].append(author['authorId'])


In [65]:
## Load Author

with open(f'/Users/elnararb/Documents/UPC/Semantic Data Management/data/authors_details.json','r') as f:
    author_details = json.load(f)



In [66]:
len(author_details)

111180

In [67]:
h_index_lst = []

for author in author_details:
    if author['hIndex'] != None:
        h_index_lst.append(author['hIndex'])
df = pd.DataFrame(h_index_lst,columns=['h_index'])

In [68]:
df.describe()

,h_index
count,111180.000000
mean,13.798660
std,19.030681
min,0.000000
25%,2.000000
50%,7.000000
75%,18.000000
max,298.000000


In [69]:
h_index_threshold = 5

In [70]:
author_h_index = {}

for author in author_details:
    if author['hIndex'] != None:
        author_h_index[author['authorId']] = author['hIndex']
    else:
        author_h_index[author['authorId']] = 0

In [71]:
paper_reviewers = {}
paper_with_no_reviewers = []

for paper in paper_lst:

    paper_author = set([author['authorId'] for author in paper['authors']])

    
    if paper['fieldsOfStudy'] != None:
        selected_author = []
        for domain in paper['fieldsOfStudy'] :
            selected_author.extend(domain_author[domain])
        
        selected_author = list(set(selected_author))

        selected_author = set([ids for ids in selected_author if ids != None and ids in author_h_index.keys() and author_h_index[ids]>=h_index_threshold])

        choosen_authors = list(selected_author.difference(paper_author))


        if len(choosen_authors)>=3:

            num_ids = random.choice([2,3])
            selected_ids = random.sample(choosen_authors,num_ids)

            paper_reviewers[paper['paperId']] = selected_ids
    
        elif len(choosen_authors)>0:
            paper_reviewers[paper['paperId']] = selected_ids
        else:
            paper_with_no_reviewers.append(paper['paperId'])
    else:
        paper_with_no_reviewers.append(paper['paperId'])

In [72]:
print(len(paper_reviewers),len(paper_with_no_reviewers))

14224 975


In [79]:
with open("/Users/elnararb/Documents/UPC/Semantic Data Management/data/paper_reviewers.json","w") as f:
    json.dump(paper_reviewers, f)

## Assigning Keywords to each paper

In [ ]:
# alex_info_lst = []

# for paper in tqdm(paper_lst):
#    try:  
#       if len(paper['externalIds']) > 0:
#          if 'DOI' in paper['externalIds']:
#             doi = paper['externalIds']['DOI']
#             doi_url = f"https://api.openalex.org/works/https://doi.org/{doi}"
#             response_url = requests.get(doi_url).json()
#             response_url['paperId'] = paper['paperId']
#             alex_info_lst.append(response_url)   
#          elif 'MAG' in paper['externalIds']:
#             mag = paper['externalIds']['MAG']
#             mag_url = f"https://api.openalex.org/works?filter=ids.mag:{mag}"
#             response_url = requests.get(mag_url).json()
#             response_url['paperId'] = paper['paperId']
#             alex_info_lst.append(response_url)  

#    except Exception as e:
#       print(f'error as {e}')

  7%|▋         | 330/4912 [02:26<33:59,  2.25it/s]

error as Expecting value: line 1 column 1 (char 0)


  7%|▋         | 347/4912 [02:34<31:00,  2.45it/s]

error as Expecting value: line 1 column 1 (char 0)


 11%|█         | 518/4912 [03:48<30:30,  2.40it/s]

...

 97%|█████████▋| 4758/4912 [36:31<01:09,  2.21it/s]

error as Expecting value: line 1 column 1 (char 0)


 98%|█████████▊| 4829/4912 [37:03<00:39,  2.11it/s]

error as Expecting value: line 1 column 1 (char 0)


100%|██████████| 4912/4912 [37:41<00:00,  2.17it/s]


In [20]:
len(alex_info_lst)

4827

In [ ]:
# with open('/Users/elnararb/Documents/UPC/Semantic Data Management/data/alex_info_new.json','w') as f:
#     json.dump(alex_info_lst,f)

In [60]:
import json
# 4827, 17036
def append_papers(new_papers, existing_file):
    # Load existing papers
    count=0
    try:
        with open(existing_file, 'r') as f:
            existing_data = json.load(f)
    except FileNotFoundError:
        existing_data = []
    len_before=len(existing_data)
    # Create a set of existing paper IDs for fast lookup
    existing_ids = {p['paperId'] for p in existing_data}

    # Add new papers only if they don't exist
    for paper in new_papers:
        paper_id = paper['paperId']  # or whatever your unique ID field is
        if paper_id not in existing_ids:
            existing_data.append(paper)
            existing_ids.add(paper_id)  # Update the set
            count+=1
    # Save back to file
    with open(existing_file, 'w') as f:
        json.dump(existing_data, f, indent=2)

    print(f"Added {count} to file with {len_before} and final count {len(existing_data)}")  # Return total count

def append_authors(new_authors, existing_file):
    # Load existing papers
    count=0
    try:
        with open(existing_file, 'r') as f:
            existing_data = json.load(f)
    except FileNotFoundError:
        existing_data = []
    len_before=len(existing_data)
    # Create a set of existing paper IDs for fast lookup
    existing_ids = {p['authorId'] for p in existing_data}

    # Add new papers only if they don't exist
    for paper in new_authors:
        author_id = paper['authorId']  # or whatever your unique ID field is
        if author_id not in existing_ids:
            existing_data.append(paper)
            existing_ids.add(author_id)  # Update the set
            count+=1
    # Save back to file
    with open(existing_file, 'w') as f:
        json.dump(existing_data, f, indent=2)

    print(f"Added {count} to file with {len_before} and final count {len(existing_data)}")  # Return total count

In [25]:
with open(f'/Users/elnararb/Documents/UPC/Semantic Data Management/data/alex_info_new.json','r') as f:
    new_data = json.load(f)
append_papers(new_data, '/Users/elnararb/Documents/UPC/Semantic Data Management/data/alex_info.json')

21847

In [32]:
with open(f'/Users/elnararb/Documents/UPC/Semantic Data Management/data/paper_details_new/conference_papers_citations.json','r') as f:
    new_data = json.load(f)
append_papers(new_data, '/Users/elnararb/Documents/UPC/Semantic Data Management/data/paper_details/conference_papers_citations.json')

Added 1927 to file with 2493 and final count 4420


In [62]:
with open(f'/Users/elnararb/Documents/UPC/Semantic Data Management/data/authors_details_new.json','r') as f:
    new_data = json.load(f)
append_authors(new_data, '/Users/elnararb/Documents/UPC/Semantic Data Management/data/authors_details.json')

Added 24983 to file with 86197 and final count 111180


In [7]:
alex_info_lst=[]
with open(f'/Users/elnararb/Documents/UPC/Semantic Data Management/data/alex_info.json','r') as f:
    data = json.load(f)
alex_info_lst.extend(data)

In [8]:
len(alex_info_lst)

21847

In [9]:
alex_info_dict = {}

for paper in alex_info_lst:

    if 'keywords' in paper.keys():
        alex_info_dict[paper['paperId']] =  [x['display_name'] for x in paper['keywords'] if x['display_name']!='Plain Text']

In [10]:
domain = {
    "Computer Science":["databases","transformer","encryption","mining", "data management", "indexing", "data modeling", "big data", 
          "data processing", "data storage", "data querying"],
    "Medicine":["diagnosis","health","disease","Genomic"],
    "Biology": ["genome","Epigenomic","phylogenetics","receptor"],
    "Chemistry":["synthesis","Bioinorganic","Molecular","Chromatographic"],
    "Mathematics":["curves","probability","Harmonic","hyperbolic"]
}

In [11]:
paper_keywords = {}

for paper in paper_lst:
    initial_keywords = {kw.lower(): kw for kw in alex_info_dict.get(paper['paperId'], [])}
    
    if paper['fieldsOfStudy'] is not None:
        for field in paper['fieldsOfStudy']:
            if field in domain:
                for word in domain[field]:
                    if word.lower() in paper['title'].lower():
                        initial_keywords[word.lower()] = word  # Overwrites with last seen case
    
    paper_keywords[paper['paperId']] = list(initial_keywords.values())

# paper_keywords = {}

# for paper in paper_lst:

#     initial_lst = []
#     if paper['paperId'] in alex_info_dict.keys() :
#         initial_lst = alex_info_dict[paper['paperId']]
    
    
#     if paper['fieldsOfStudy']!=None:
#         for field in paper['fieldsOfStudy']:
#             if field in domain.keys():  
#                 for word in domain[field]:
#                     if word.lower() in paper['title'].lower():
#                         initial_lst.append(word)

#     paper_keywords[paper['paperId']] = list(set(initial_lst))

In [12]:
len(paper_keywords)

15199

In [13]:
with open("/Users/elnararb/Documents/UPC/Semantic Data Management/data/paper_keywords.json",'w') as f:
    json.dump(paper_keywords,f)

## Proceedings and Venues

In [80]:
from faker import Faker
import random
fake = Faker()

In [81]:
def generate_venue():
    city = fake.city()
    country = fake.country()
    institution = fake.random_element(elements=(
        "University of " + fake.city(),
        fake.company() + " Research Center",
        "International " + fake.word().title() + " Institute"
    ))
    return {
        "city": city,
        "country": country,
        "institution": institution
    }

In [11]:
conference_articles = []

for paper in paper_lst:
    
    if paper['publicationTypes'] != None and  "Conference" in paper['publicationTypes']: 
        conference_articles.append(paper)

    # elif "conference" in paper['title'].lower():
    #     conference_articles.append(paper)
        
        

NameError: name 'paper_lst' is not defined

In [12]:
import pandas as pd
conference_df = pd.DataFrame(conference_articles)

In [84]:
conference_df.head()

,paperId,externalIds,corpusId,publicationVenue,url,title,abstract,venue,year,referenceCount,...,fieldsOfStudy,s2FieldsOfStudy,tldr,publicationTypes,publicationDate,journal,citationStyles,authors,citations,references
0,1fad26186df2927fa46a552802942523e429839a,"{'DBLP': 'conf/trustcom/MaLG21', 'DOI': '10.11...",247321770,"{'id': 'ad34e82f-f936-47a5-9c78-c4453dff3f60',...",https://www.semanticscholar.org/paper/1fad2618...,Improving the Effect of Frequent Itemset Minin...,Frequent itemset mining is a basic data mining...,"International Conference on Trust, Security an...",2021.0,23,...,[Computer Science],"[{'category': 'Computer Science', 'source': 'e...","{'model': 'tldr@v2.0.0', 'text': 'This paper u...","[JournalArticle, Conference]",2021-10-01,{'name': '2021 IEEE 20th International Confere...,"{'bibtex': '@Article{Ma2021ImprovingTE, autho...","[{'authorId': '2409293', 'name': 'Xuebin Ma'},...",[],[]
1,ebe53532339772db4d79d053f7be8e385a4cd58b,"{'ArXiv': '2410.13458', 'DBLP': 'journals/corr...",273404326,"{'id': '41bf9ed3-85b3-4c90-b015-150e31690253',...",https://www.semanticscholar.org/paper/ebe53532...,MedINST: Meta Dataset of Biomedical Instructions,The integration of large language model (LLM) ...,Conference on Empirical Methods in Natural Lan...,2024.0,61,...,[Computer Science],"[{'category': 'Computer Science', 'source': 'e...","{'model': 'tldr@v2.0.0', 'text': 'This work in...","[JournalArticle, Conference]",2024-10-17,"{'name': 'ArXiv', 'volume': 'abs/2410.13458'}","{'bibtex': '@Article{Han2024MedINSTMD, author...","[{'authorId': '2326674128', 'name': 'Wenhan Ha...",[],[]
2,d58d863a4d03e0a4ffadd6780c65572e3a7c420c,"{'DBLP': 'conf/icacci/MaradithayaS17', 'MAG': ...",2884602,"{'id': 'ec37ab61-bda3-4666-b030-93e91de580dd',...",https://www.semanticscholar.org/paper/d58d863a...,Secure model for clustering distributed data,Data of similar nature are disseminated across...,International Conference on Advances in Comput...,2017.0,22,...,[Computer Science],"[{'category': 'Computer Science', 'source': 'e...","{'model': 'tldr@v2.0.0', 'text': 'The proposed...","[JournalArticle, Conference]",2017-09-01,{'name': '2017 International Conference on Adv...,"{'bibtex': '@Article{Maradithaya2017SecureMF, ...","[{'authorId': '32733176', 'name': 'Sumana Mara...",[],[]
3,8f4d381eff8a33b20c5ec4afa6cc133c82ab441f,"{'DBLP': 'conf/compsac/NakagawaHKFS14', 'MAG':...",9733326,None,https://www.semanticscholar.org/paper/8f4d381e...,m-cloud -- Distributed Statistical Computation...,"In the age of IoT (Internet of Things), tens (...",2014 IEEE 38th International Computer Software...,2014.0,11,...,[Computer Science],"[{'category': 'Computer Science', 'source': 'e...","{'model': 'tldr@v2.0.0', 'text': 'This work pr...","[JournalArticle, Conference]",2014-07-21,{'name': '2014 IEEE 38th International Compute...,"{'bibtex': '@Article{Nakagawa2014mcloudD, aut...","[{'authorId': '32845436', 'name': 'Ikuo Nakaga...",[],[]
4,7c95481dd12d7da780b4a1bff4eb61aec96031ef,"{'MAG': '2541088479', 'DBLP': 'conf/socialcom/...",796272,None,https://www.semanticscholar.org/paper/7c95481d...,Achieving Full Security in Privacy-Preserving ...,"In privacy-preserving data mining, a number of...","2011 IEEE Third Int'l Conference on Privacy, S...",2011.0,26,...,[Computer Science],"[{'category': 'Computer Science', 'source': 'e...","{'model': 'tldr@v2.0.0', 'text': 'The authors ...","[JournalArticle, Conference]",2011-10-01,{'name': '2011 IEEE Third Int'l Conference on ...,"{'bibtex': '@Article{Blanton2011AchievingFS, ...","[{'authorId': '145435850', 'name': 'Marina Bla...",[],[]


In [85]:
sub_df = conference_df[['paperId','title','publicationDate','year','venue']]
sub_df['venue_id'] = conference_df['publicationVenue'].apply(lambda x: x.get('id', None) if x else None)
sub_df['pages'] = conference_df['journal'].apply(lambda x: x.get('pages', None) if x else None)

/var/folders/0r/x2rt66gj2h75vp5rrm7772br0000gn/T/ipykernel_5088/1133201505.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['venue_id'] = conference_df['publicationVenue'].apply(lambda x: x.get('id', None) if x else None)
/var/folders/0r/x2rt66gj2h75vp5rrm7772br0000gn/T/ipykernel_5088/1133201505.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['pages'] = conference_df['journal'].apply(lambda x: x.get('pages', None) if x else None)


In [86]:
sub_df.head()

,paperId,title,publicationDate,year,venue,venue_id,pages
0,1fad26186df2927fa46a552802942523e429839a,Improving the Effect of Frequent Itemset Minin...,2021-10-01,2021.0,"International Conference on Trust, Security an...",ad34e82f-f936-47a5-9c78-c4453dff3f60,436-443
1,ebe53532339772db4d79d053f7be8e385a4cd58b,MedINST: Meta Dataset of Biomedical Instructions,2024-10-17,2024.0,Conference on Empirical Methods in Natural Lan...,41bf9ed3-85b3-4c90-b015-150e31690253,None
2,d58d863a4d03e0a4ffadd6780c65572e3a7c420c,Secure model for clustering distributed data,2017-09-01,2017.0,International Conference on Advances in Comput...,ec37ab61-bda3-4666-b030-93e91de580dd,1256-1260
3,8f4d381eff8a33b20c5ec4afa6cc133c82ab441f,m-cloud -- Distributed Statistical Computation...,2014-07-21,2014.0,2014 IEEE 38th International Computer Software...,None,301-305
4,7c95481dd12d7da780b4a1bff4eb61aec96031ef,Achieving Full Security in Privacy-Preserving ...,2011-10-01,2011.0,"2011 IEEE Third Int'l Conference on Privacy, S...",None,925-934


In [113]:
sub_df[sub_df['year'].isnull()]

,paperId,title,publicationDate,year,venue,venue_id,pages,proceedings,location,new_conference_name


In [111]:
# sub_df.loc[1120,'year'] = 2020
# sub_df.loc[994,'year'] = 2017
sub_df.loc[1020,'year'] = 2023
sub_df.loc[1331,'year'] = 2023

In [112]:
sub_df.isnull().sum()

paperId                   0
title                     0
publicationDate         124
year                      0
venue                     0
venue_id               1145
pages                   430
proceedings            1659
location               2620
new_conference_name       0
dtype: int64

In [114]:
conf_name = list(sub_df['venue'].value_counts().to_dict().keys())

In [115]:
sub_df[sub_df['venue']=='International Conference on Computing Communication and Networking Technologies']['year'].value_counts()

year
2024.0    34
2023.0     7
2019.0     4
2020.0     2
2022.0     2
2018.0     1
2021.0     1
Name: count, dtype: int64

In [116]:
sub_df['proceedings'] = np.nan
sub_df['location'] = np.nan

/var/folders/0r/x2rt66gj2h75vp5rrm7772br0000gn/T/ipykernel_5088/4082265470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['proceedings'] = np.nan
/var/folders/0r/x2rt66gj2h75vp5rrm7772br0000gn/T/ipykernel_5088/4082265470.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['location'] = np.nan


In [117]:
from datetime import datetime

# Dictionary to map textual ordinals to numbers (expand as needed)
ORDINAL_WORDS = {
    'first': 1, 'second': 2, 'third': 3, 'fourth': 4, 'fifth': 5, 'sixth': 6,
    'seventh': 7, 'eighth': 8, 'ninth': 9, 'tenth': 10, 'eleventh': 11,
    'twelfth': 12, 'thirteenth': 13, 'fourteenth': 14, 'fifteenth': 15,
    'sixteenth': 16, 'seventeenth': 17, 'eighteenth': 18, 'nineteenth': 19,
    'twentieth': 20, 'twentyfirst': 21, 'twentysecond': 22, 'twentythird': 23,
    'twentyfourth': 24, 'twentyfifth': 25, 'twentysixth': 26, 'twentyseventh': 27,
    'twentyeighth': 28, 'twentyninth': 29, 'thirtieth': 30
}

def extract_edition(conference_name):
    pattern = re.compile(
        r"""
        (\b\d{4}\b)|                        # 4-digit year (group 1)
        (\d+)(?:st|nd|rd|th)\b|             # Numeric ordinal (group 2)
        Edition\s+(\d+)|                    # Explicit "Edition X" (group 3)
        ['’](\d{2})\b|                      # Apostrophe year (group 4)
        \b(First|Second|Third|Fourth|Fifth|Sixth|Seventh|Eighth|Ninth|Tenth|
        Eleventh|Twelfth|Thirteenth|Fourteenth|Fifteenth|Sixteenth|Seventeenth|
        Eighteenth|Nineteenth|Twentieth|Twenty[- ]?First|Twenty[- ]?Second|
        Twenty[- ]?Third|Twenty[- ]?Fourth|Twenty[- ]?Fifth|Twenty[- ]?Sixth|
        Twenty[- ]?Seventh|Twenty[- ]?Eighth|Twenty[- ]?Ninth|Thirtieth)\b
        """,
        re.IGNORECASE | re.X
    )

    editions = []
    for match in pattern.finditer(conference_name):
        year_4d = match.group(1)
        numeric_ordinal = match.group(2)
        edition_x = match.group(3)
        apostrophe_year = match.group(4)
        textual_ordinal = match.group(5)

        if year_4d:
            editions.append(('year', int(year_4d)))
        elif numeric_ordinal:
            editions.append(('ordinal', int(numeric_ordinal)))
        elif edition_x:
            editions.append(('edition_x', int(edition_x)))
        elif apostrophe_year:
            year = int(apostrophe_year)
            current_year = datetime.now().year
            full_year = 2000 + year if year < (current_year - 2000 + 1) else 1900 + year
            editions.append(('apostrophe_year', full_year))
        elif textual_ordinal:
            key = textual_ordinal.lower().replace('-', '').replace(' ', '')
            editions.append(('textual_ordinal', ORDINAL_WORDS.get(key, None)))

    # Priority order: edition_x > textual_ordinal > ordinal > apostrophe_year > year
    priority_order = ['edition_x', 'textual_ordinal', 'ordinal', 'apostrophe_year', 'year']
    for category in priority_order:
        for ed_type, value in editions:
            if ed_type == category and value is not None:
                return value
    return None


In [118]:
def clean_conference_name(conference_name):
    # Remove edition-related terms to get the base name
    pattern = re.compile(
        r"""
        \b\d{4}\b|                        # 4-digit year
        \d+(?:st|nd|rd|th)\b|             # Numeric ordinal
        Edition\s+\d+|                    # Explicit "Edition X"
        ['’]\d{2}\b|                      # Apostrophe year
        \b(?:First|Second|Third|Fourth|Fifth|Sixth|Seventh|Eighth|Ninth|Tenth|
        Eleventh|Twelfth|Thirteenth|Fourteenth|Fifteenth|Sixteenth|Seventeenth|
        Eighteenth|Nineteenth|Twentieth|Twenty[- ]?First|Twenty[- ]?Second|
        Twenty[- ]?Third|Twenty[- ]?Fourth|Twenty[- ]?Fifth|Twenty[- ]?Sixth|
        Twenty[- ]?Seventh|Twenty[- ]?Eighth|Twenty[- ]?Ninth|Thirtieth)\b
        """,
        re.IGNORECASE | re.X
    )
    cleaned = pattern.sub('', conference_name)
    # Clean up extra spaces and punctuation
    cleaned = re.sub(r'\s{2,}', ' ', cleaned)  # Replace multiple spaces
    cleaned = re.sub(r'[ ,\-]+$', '', cleaned)  # Remove trailing punctuation
    cleaned = re.sub(r'^[ ,\-]+', '', cleaned)  # Remove leading punctuation
    return cleaned.strip()

In [119]:
sub_df['proceedings'] = sub_df['venue'].apply(extract_edition)
sub_df['new_conference_name'] = sub_df['venue'].apply(clean_conference_name)

/var/folders/0r/x2rt66gj2h75vp5rrm7772br0000gn/T/ipykernel_5088/3743250433.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['proceedings'] = sub_df['venue'].apply(extract_edition)
/var/folders/0r/x2rt66gj2h75vp5rrm7772br0000gn/T/ipykernel_5088/3743250433.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['new_conference_name'] = sub_df['venue'].apply(clean_conference_name)


In [120]:
df1 = sub_df[~sub_df['proceedings'].isnull()]
df1 = df1.reset_index(drop=True)

In [121]:
df2 = sub_df[sub_df['proceedings'].isnull()]
df2 = df2.reset_index(drop=True)

In [122]:
grouped = df1.groupby(by=['new_conference_name','proceedings'])

In [123]:
df2[df2['venue_id'].notna()]

,paperId,title,publicationDate,year,venue,venue_id,pages,proceedings,location,new_conference_name
0,1fad26186df2927fa46a552802942523e429839a,Improving the Effect of Frequent Itemset Minin...,2021-10-01,2021.0,"International Conference on Trust, Security an...",ad34e82f-f936-47a5-9c78-c4453dff3f60,436-443,NaN,NaN,"International Conference on Trust, Security an..."
1,ebe53532339772db4d79d053f7be8e385a4cd58b,MedINST: Meta Dataset of Biomedical Instructions,2024-10-17,2024.0,Conference on Empirical Methods in Natural Lan...,41bf9ed3-85b3-4c90-b015-150e31690253,None,NaN,NaN,Conference on Empirical Methods in Natural Lan...
2,d58d863a4d03e0a4ffadd6780c65572e3a7c420c,Secure model for clustering distributed data,2017-09-01,2017.0,International Conference on Advances in Comput...,ec37ab61-bda3-4666-b030-93e91de580dd,1256-1260,NaN,NaN,International Conference on Advances in Comput...
3,37ad55166ec7f279a8065b2506ac95cbea1647e9,A Survey on Privacy Preserving Association Rul...,2018-07-01,2018.0,International Conference on Information Scienc...,906a26c6-9eef-4b5d-b6d5-6dcb37e21fc2,93-97,NaN,NaN,International Conference on Information Scienc...
4,29a6fc9ac58ee194b2c9e4301a2fca7b8f92508a,Density Peaks Clustering with Differential Pri...,None,2017.0,Conference on Innovative Data Systems Research,528ced1f-e949-4e1a-8fee-2ffbf0be551d,None,NaN,NaN,Conference on Innovative Data Systems Research
...,...,...,...,...,...,...,...,...,...,...
1654,2629d5bc29ce83ad553906e63b07a172d9046817,Geospatial data storage based on HBase and Map...,2017-08-01,2017.0,International Conference on Agro-Geoinformatics,4bbf5bf8-63dd-44c7-a46b-00949c8026bd,1-4,NaN,NaN,International Conference on Agro-Geoinformatics
1655,63c0b9cb7a8c3faf81248d43e62d0ba754744a85,Collecting and Mining Big Data for Electric Ve...,2015-04-13,2015.0,International Conference on Information Techno...,67128bf0-cf99-46bd-8915-1d9a05820d0a,626-631,NaN,NaN,International Conference on Information Techno...
1656,58d07db4ec182b6f5a22d907c9ea2e7098bf9f59,A framework to model big data driven complex c...,2014-10-27,2014.0,International Conference on Automation and Com...,28a83bf1-b6e5-416f-8c5a-fdc53e54f3a1,283-288,NaN,NaN,International Conference on Automation and Com...
1657,6bf808c3cd762b850f73b6aa735f9c3821cad7cc,Data Partitioning Scheme for Efficient Distrib...,2019-03-11,2019.0,International Computer Science Conference,6bf48d72-eba2-4918-b9a0-153280062513,24-31,NaN,NaN,International Computer Science Conference


In [124]:
for g in grouped.groups:
    index = list(grouped.groups[g])
    df1.loc[index,'location'] = fake.city()

/var/folders/0r/x2rt66gj2h75vp5rrm7772br0000gn/T/ipykernel_5088/3701494661.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'East Jonathan' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df1.loc[index,'location'] = fake.city()


In [125]:
grouped = df2.groupby(by=['year','new_conference_name'])

In [126]:
conf_count = {}

for g in grouped.groups:
    index = list(grouped.groups[g])
    
    df2.loc[index,'location'] = fake.city()
    
    if g[1] in conf_count.keys():
        conf_count[g[1]] += 1
        df2.loc[index,'proceedings'] = conf_count[g[1]]
    else:
        conf_count[g[1]] = 3
        df2.loc[index,'proceedings'] = 3
        

/var/folders/0r/x2rt66gj2h75vp5rrm7772br0000gn/T/ipykernel_5088/3967363529.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Port Kristinaton' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df2.loc[index,'location'] = fake.city()


In [127]:
df2['new_conference_name'].value_counts()

new_conference_name
IEEE International Conference on Data Engineering                                           88
Journal of Physics: Conference Series                                                       61
International Conference on Computing Communication and Networking Technologies             51
SIGMOD Conference                                                                           43
International Conference on Information and Knowledge Management                            28
                                                                                            ..
International Conference on Software Technology and Engineering                              1
International Conference on Software, Knowledge, Information Management and Applications     1
International Conference on Advanced Communication Control and Computing Technologies        1
International Conference on Computer Vision and Data Mining                                  1
International Conference on Ub

In [128]:
final_df = pd.concat([df1,df2])

In [129]:
final_df = final_df.reset_index(drop=True)
final_df=final_df.drop_duplicates()

In [130]:
len(final_df[final_df['venue_id'].notna()])

1475

In [131]:
final_df = final_df[['paperId','venue_id','year','proceedings','location', 'pages','new_conference_name']]
final_df.dropna(subset=['venue_id'], inplace=True)
final_df=final_df.rename(columns={'proceedings': 'edition'})

In [132]:
final_df.head()

,paperId,venue_id,year,edition,location,pages,new_conference_name
961,1fad26186df2927fa46a552802942523e429839a,ad34e82f-f936-47a5-9c78-c4453dff3f60,2021.0,4.0,Lopezshire,436-443,"International Conference on Trust, Security an..."
962,ebe53532339772db4d79d053f7be8e385a4cd58b,41bf9ed3-85b3-4c90-b015-150e31690253,2024.0,7.0,West Dianefurt,None,Conference on Empirical Methods in Natural Lan...
963,d58d863a4d03e0a4ffadd6780c65572e3a7c420c,ec37ab61-bda3-4666-b030-93e91de580dd,2017.0,5.0,East Michaelfurt,1256-1260,International Conference on Advances in Comput...
964,37ad55166ec7f279a8065b2506ac95cbea1647e9,906a26c6-9eef-4b5d-b6d5-6dcb37e21fc2,2018.0,3.0,East Stephen,93-97,International Conference on Information Scienc...
965,29a6fc9ac58ee194b2c9e4301a2fca7b8f92508a,528ced1f-e949-4e1a-8fee-2ffbf0be551d,2017.0,3.0,Port Bryan,None,Conference on Innovative Data Systems Research


In [133]:
final_df.to_csv('/Users/elnararb/Documents/UPC/Semantic Data Management/data/paper_proceedings_location_new.csv',index=False)

## Creating Metadata for Reviews

In [134]:
peer_reviews = [
    "The study addresses an important gap, but the small sample size limits generalizability. Recommend major revisions.",
    "Methodology is robust, but the discussion lacks recent literature. Include more citations before acceptance.",
    "Theoretical framework is strong, but empirical validation is missing. Add case studies for clarity.",
    "Results are compelling, but statistical analysis requires deeper justification. Revise and resubmit.",
    "Innovative topic, but the writing is unclear in sections 2 and 4. Improve flow and terminology.",
    "Data presentation is excellent, but conclusions overreach the findings. Tone down claims.",
    "Strong interdisciplinary approach, but figures lack labels. Redesign visuals for better readability.",
    "Well-structured argument, but lacks engagement with counterarguments. Expand the literature review.",
    "Practical implications are significant, but ethical considerations are absent. Address in revision.",
    "Original contribution to the field, but the abstract needs simplification. Clarify key takeaways."
]
review_decisions = ['accepted', 'rejected']


In [135]:
import json
with open("/Users/elnararb/Documents/UPC/Semantic Data Management/data/paper_reviewers.json",'r') as f:
    paper_reviewers = json.load(f)

In [136]:
import random
import uuid
paper_reviewer_metadata = {}

for paper_id, author_ids in paper_reviewers.items():
    num_reviewers = len(author_ids)
    
    for i, author_id in enumerate(author_ids, 1):
        if paper_id not in paper_reviewer_metadata:
            paper_reviewer_metadata[paper_id] = {}
        review_id=str(uuid.uuid4())
        paper_reviewer_metadata[paper_id][f"reviewer_{i}"] = {
            "reviewId": review_id,
            "authorId": author_id,
            "comments": random.choice(peer_reviews),
            "vote": random.choices(review_decisions, weights=[0.8, 0.2])[0],  # 80% accept, 20% reject
        }

In [137]:
with open("/Users/elnararb/Documents/UPC/Semantic Data Management/data/paper_reviewers_metadata.json",'w') as f:
    json.dump(paper_reviewer_metadata,f)

# My part

In [4]:
import json
import os
import pandas as pd

with open('/Users/elnararb/Documents/UPC/Semantic Data Management/data/alex_info.json', 'r') as file:
    nested_data = json.load(file)[:5] 

df = pd.json_normalize(nested_data)
df.head()
#print(nested_data)


,id,doi,title,display_name,publication_year,publication_date,language,type,type_crossref,indexed_in,...,abstract_inverted_index.applied,abstract_inverted_index.correctly,abstract_inverted_index.working,abstract_inverted_index.learning,abstract_inverted_index.explanation,abstract_inverted_index.also,abstract_inverted_index.given,abstract_inverted_index.how,abstract_inverted_index.why,abstract_inverted_index.apply
0,https://openalex.org/W3037166189,https://doi.org/10.3233/faia200334,Transformer-Based Argument Mining for Healthca...,Transformer-Based Argument Mining for Healthca...,2020,2020-01-01,en,book-chapter,book-chapter,[crossref],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://openalex.org/W2940603449,https://doi.org/10.1109/iccubea.2018.8697353,Privacy-Preserving Outsourced Mining of D-Ecla...,Privacy-Preserving Outsourced Mining of D-Ecla...,2018,2018-08-01,en,article,proceedings-article,[crossref],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://openalex.org/W4200206740,https://doi.org/10.1109/embc46164.2021.9629552,Transformer-based CNNs: Mining Temporal Contex...,Transformer-based CNNs: Mining Temporal Contex...,2021,2021-11-01,en,article,proceedings-article,"[crossref, pubmed]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://openalex.org/W2991708894,https://doi.org/10.1109/icems.2019.8922194,Noise and Vibration Analysis of Dry-Type Power...,Noise and Vibration Analysis of Dry-Type Power...,2019,2019-08-01,en,article,proceedings-article,[crossref],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://openalex.org/W3010727636,https://doi.org/10.1109/eiconrus49466.2020.903...,Order-preserving Encryption as a Tool for Priv...,Order-preserving Encryption as a Tool for Priv...,2020,2020-01-01,en,article,proceedings-article,[crossref],...,[126],[129],[130],[133],[136],[138],[139],[140],[142],[147]


In [6]:
pd.set_option('display.max_columns', None)
df.head()
# df[df['keywords'].str.contains('slco1b1', case=False, na=False)]

,id,doi,title,display_name,publication_year,publication_date,language,type,type_crossref,indexed_in,authorships,institution_assertions,countries_distinct_count,institutions_distinct_count,corresponding_author_ids,corresponding_institution_ids,apc_list,apc_paid,fwci,has_fulltext,cited_by_count,is_retracted,is_paratext,topics,keywords,concepts,mesh,locations_count,locations,sustainable_development_goals,grants,datasets,versions,referenced_works_count,referenced_works,related_works,abstract_inverted_index_v3,cited_by_api_url,counts_by_year,updated_date,created_date,paperId,ids.openalex,ids.doi,ids.mag,primary_location.is_oa,primary_location.landing_page_url,primary_location.pdf_url,primary_location.source.id,primary_location.source.display_name,primary_location.source.issn_l,primary_location.source.issn,primary_location.source.is_oa,primary_location.source.is_in_doaj,primary_location.source.is_indexed_in_scopus,primary_location.source.is_core,primary_location.source.host_organization,primary_location.source.host_organization_name,primary_location.source.host_organization_lineage,primary_location.source.host_organization_lineage_names,primary_location.source.type,primary_location.license,primary_location.license_id,primary_location.version,primary_location.is_accepted,primary_location.is_published,open_access.is_oa,open_access.oa_status,open_access.oa_url,open_access.any_repository_has_fulltext,citation_normalized_percentile.value,citation_normalized_percentile.is_in_top_1_percent,citation_normalized_percentile.is_in_top_10_percent,cited_by_percentile_year.min,cited_by_percentile_year.max,biblio.volume,biblio.issue,biblio.first_page,biblio.last_page,primary_topic.id,primary_topic.display_name,primary_topic.score,primary_topic.subfield.id,primary_topic.subfield.display_name,primary_topic.field.id,primary_topic.field.display_name,primary_topic.domain.id,primary_topic.domain.display_name,best_oa_location.is_oa,best_oa_location.landing_page_url,best_oa_location.pdf_url,best_oa_location.source.id,best_oa_location.source.display_name,best_oa_location.source.issn_l,best_oa_location.source.issn,best_oa_location.source.is_oa,best_oa_location.source.is_in_doaj,best_oa_location.source.is_indexed_in_scopus,best_oa_location.source.is_core,best_oa_location.source.host_organization,best_oa_location.source.host_organization_name,best_oa_location.source.host_organization_lineage,best_oa_location.source.host_organization_lineage_names,best_oa_location.source.type,best_oa_location.license,best_oa_location.license_id,best_oa_location.version,best_oa_location.is_accepted,best_oa_location.is_published,abstract_inverted_index.Argument(ation),abstract_inverted_index.Mining,abstract_inverted_index.(AM),abstract_inverted_index.typically,abstract_inverted_index.aims,abstract_inverted_index.at,abstract_inverted_index.identifying,abstract_inverted_index.argumentative,abstract_inverted_index.components,abstract_inverted_index.in,abstract_inverted_index.text,abstract_inverted_index.and,abstract_inverted_index.predicting,abstract_inverted_index.the,abstract_inverted_index.relations,abstract_inverted_index.among,abstract_inverted_index.them.,abstract_inverted_index.Evidence-based,abstract_inverted_index.decision,abstract_inverted_index.making,abstract_inverted_index.health-care,abstract_inverted_index.domain,abstract_inverted_index.targets,abstract_inverted_index.supporting,abstract_inverted_index.clinicians,abstract_inverted_index.their,abstract_inverted_index.deliberation,abstract_inverted_index.process,abstract_inverted_index.to,abstract_inverted_index.establish,abstract_inverted_index.best,abstract_inverted_index.course,abstract_inverted_index.of,abstract_inverted_index.action,abstract_inverted_index.for,abstract_inverted_index.case,abstract_inverted_index.under,abstract_inverted_index.evaluation.,abstract_inverted_index.Although,abstract_inverted_index.reasoning,abstract_inverted_index.stage,abstract_inverted_index.this,abstract_inverted_index.kind,abstract_

In [51]:
import json
import os

# Directory containing JSON files
json_dir = '/Users/elnararb/Documents/UPC/Semantic Data Management/data/paper_details'

# List to hold combined data
combined_papers_data = []
seen_ids = set()

# Loop through each JSON file in the directory
for filename in os.listdir(json_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(json_dir, filename)
        with open(file_path, 'r') as file:
            data = json.load(file)  # Load JSON data
            # Filter data
            filtered_data = [
                paper for paper in data
                if paper is not None and paper.get("publicationTypes") is not None and ("JournalArticle" in paper.get("publicationTypes", [])
                    or "Conference" in paper.get("publicationTypes", []))
            ]
            for paper in filtered_data:
                paper_id = paper.get("paperId")
                journal = paper.get("journal", [])
                if "JournalArticle" in paper.get("publicationTypes", []) and journal is None and "Conference" not in paper.get("publicationTypes", []):
                    continue
                if "JournalArticle" in paper.get("publicationTypes", []) and "Conference" not in paper.get("publicationTypes", []) and (journal.get("volume") is None or journal.get("volume")==""):
                    continue
                if paper_id and paper_id not in seen_ids:
                    seen_ids.add(paper_id)
                    combined_papers_data.append(paper)

# Write combined data to a new JSON file
with open('/Users/elnararb/Documents/UPC/Semantic Data Management/data/combined_papers_data.json', 'w') as outfile:
    json.dump(combined_papers_data, outfile, indent=4)

print("JSON files combined successfully!")

JSON files combined successfully!


In [52]:
len(combined_papers_data)

15199

In [139]:
import json
import os
import pandas as pd
with open('/Users/elnararb/Documents/UPC/Semantic Data Management/data/combined_papers_data.json', 'r') as file:
    nested_data = json.load(file)

df = pd.json_normalize(nested_data)
df.head()

,paperId,corpusId,url,title,abstract,venue,year,referenceCount,citationCount,influentialCitationCount,...,journal.pages,publicationVenue,externalIds.DBLP,tldr,externalIds.ArXiv,publicationVenue.alternate_issns,externalIds.ACL,journal,openAccessPdf.license,openAccessPdf.disclaimer
0,c5447f3c64ec971b610d663f70d9e0051e3fc8d2,231668203,https://www.semanticscholar.org/paper/c5447f3c...,Catechin and curcumin interact with S protein ...,None,Scientific Reports,2021.0,43,118,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,51a47b0fcb4185e57d52c7bd0bec0aa62ae4c26c,229694061,https://www.semanticscholar.org/paper/51a47b0f...,l-Arabinose Induces the Formation of Viable No...,Vibrios spp. among other bacteria form transie...,Applied and Environmental Microbiology,2020.0,60,6,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,504efb13c5a675352c3ca19c646a52bfb0baabc1,203927391,https://www.semanticscholar.org/paper/504efb13...,Functional Genomics of the Retina to Elucidate...,The retina is the light sensitive part of the ...,International Journal of Molecular Sciences,2019.0,364,9,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b3d3500e66874b3eab75140bb85ad00696251e58,246653305,https://www.semanticscholar.org/paper/b3d3500e...,Clinical assessment of blood pressure in 60 gi...,Hypertension contributes to increased risk of ...,Clinical Endocrinology,2022.0,41,2,0,...,428 - 438,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,b41c09d98afe19dee5037342568e3f410bb949f6,272740450,https://www.semanticscholar.org/paper/b41c09d9...,"Force Fields, Quantum-Mechanical- and Molecula...",Radiopharmaceuticals are currently a key tool ...,Molecules,2024.0,72,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [140]:

null_df=df[df['publicationVenue.id'].notna()]
null2_df=df[df['publicationVenue.issn'].notna()]

print(len(df[df['publicationTypes'].apply(lambda x: 'Conference' in x)]))
print(len(null_df[null_df['publicationTypes'].apply(lambda x: 'Conference' in x)]))
print(len(null2_df[null2_df['publicationTypes'].apply(lambda x: 'Conference' in x)]))

print(len(df[df['publicationTypes'].apply(lambda x: 'JournalArticle' in x and 'Conference' not in x)]))
print(len(null_df[null_df['publicationTypes'].apply(lambda x: 'JournalArticle' in x and 'Conference' not in x)]))
print(len(null2_df[null2_df['publicationTypes'].apply(lambda x: 'JournalArticle' in x and 'Conference' not in x)]))


2620
1475
113
12579
11537
11251


In [141]:
null_df[null_df['publicationTypes'].apply(lambda x: 'Conference' in x)]

,paperId,corpusId,url,title,abstract,venue,year,referenceCount,citationCount,influentialCitationCount,...,journal.pages,publicationVenue,externalIds.DBLP,tldr,externalIds.ArXiv,publicationVenue.alternate_issns,externalIds.ACL,journal,openAccessPdf.license,openAccessPdf.disclaimer
174,1fad26186df2927fa46a552802942523e429839a,247321770,https://www.semanticscholar.org/paper/1fad2618...,Improving the Effect of Frequent Itemset Minin...,Frequent itemset mining is a basic data mining...,"International Conference on Trust, Security an...",2021.0,23,4,0,...,436-443,NaN,conf/trustcom/MaLG21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,ebe53532339772db4d79d053f7be8e385a4cd58b,273404326,https://www.semanticscholar.org/paper/ebe53532...,MedINST: Meta Dataset of Biomedical Instructions,The integration of large language model (LLM) ...,Conference on Empirical Methods in Natural Lan...,2024.0,61,0,0,...,NaN,NaN,journals/corr/abs-2410-13458,NaN,2410.13458,NaN,NaN,NaN,NaN,NaN
207,d58d863a4d03e0a4ffadd6780c65572e3a7c420c,2884602,https://www.semanticscholar.org/paper/d58d863a...,Secure model for clustering distributed data,Data of similar nature are disseminated across...,International Conference on Advances in Comput...,2017.0,22,0,0,...,1256-1260,NaN,conf/icacci/MaradithayaS17,NaN,NaN,NaN,NaN,NaN,NaN,NaN
310,37ad55166ec7f279a8065b2506ac95cbea1647e9,58669463,https://www.semanticscholar.org/paper/37ad5516...,A Survey on Privacy Preserving Association Rul...,"In recent years, privacy preserving associatio...",International Conference on Information Scienc...,2018.0,34,1,0,...,93-97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
344,29a6fc9ac58ee194b2c9e4301a2fca7b8f92508a,13718492,https://www.semanticscholar.org/paper/29a6fc9a...,Density Peaks Clustering with Differential Pri...,Density peaks clustering (DPC) is a well-known...,Conference on Innovative Data Systems Research,2017.0,23,3,0,...,NaN,NaN,conf/cidr/GuoM17,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14856,2629d5bc29ce83ad553906e63b07a172d9046817,22848391,https://www.semanticscholar.org/paper/2629d5bc...,Geospatial data storage based on HBase and Map...,Traditional relational database management sys...,International Conference on Agro-Geoinformatics,2017.0,14,6,1,...,1-4,NaN,conf/agro-geoinformatics/GaoYWZ17,NaN,NaN,NaN,NaN,NaN,None,Notice: This abstract is extracted from the op...
14858,63c0b9cb7a8c3faf81248d43e62d0ba754744a85,14003161,https://www.semanticscholar.org/paper/63c0b9cb...,Collecting and Mining Big Data for Electric Ve...,Growth of Electric vehicles (EV) starts to cha...,International Conference on Information Techno...,2015.0,43,27,0,...,626-631,NaN,conf/itng/LeeW15,NaN,NaN,NaN,NaN,NaN,None,Notice: This abstract is extracted from the op...
14861,58d07db4ec182b6f5a22d907c9ea2e7098bf9f59,17407498,https://www.semanticscholar.org/paper/58d07db4...,A framework to model big data driven complex c...,"First, big data driven cyber physical systems ...",International Conference on Automation and Com...,2014.0,26,18,0,...,283-288,NaN,conf/iconac/Zhang14a,NaN,NaN,NaN,NaN,NaN,None,Notice: This abstract is extracted from the op...
14863,6bf808c3cd762b850f73b6aa735f9c3821cad7cc,77384924,https://www.semanticscholar.org/paper/6bf808c3...,Data Partitioning Scheme for Efficient Distrib...,The rapid growth of semantic data in the form ...,International Computer Science Conference,2019.0,29,10,1,...,24-31,NaN,conf/semco/HassanB19,NaN,NaN,NaN,NaN,NaN,None,Notice: This abstract is extracted from the op...


In [142]:
new=df[df['publicationVenue.name'].str.contains('workshop', case=False, na=False)]
print(len(new[new['publicationVenue.id'].notna()]))

25


In [143]:
check_volume=df[df['journal.volume'].notna()]
check_conf_vol=check_volume[check_volume['publicationTypes'].apply(lambda x: 'Conference' in x)]
print(len(check_volume[check_volume['publicationTypes'].apply(lambda x: 'Conference' in x)]))
check_conf_vol.head()

246


,paperId,corpusId,url,title,abstract,venue,year,referenceCount,citationCount,influentialCitationCount,...,journal.pages,publicationVenue,externalIds.DBLP,tldr,externalIds.ArXiv,publicationVenue.alternate_issns,externalIds.ACL,journal,openAccessPdf.license,openAccessPdf.disclaimer
197,ebe53532339772db4d79d053f7be8e385a4cd58b,273404326,https://www.semanticscholar.org/paper/ebe53532...,MedINST: Meta Dataset of Biomedical Instructions,The integration of large language model (LLM) ...,Conference on Empirical Methods in Natural Lan...,2024.0,61,0,0,...,NaN,NaN,journals/corr/abs-2410-13458,NaN,2410.13458,NaN,NaN,NaN,NaN,NaN
344,29a6fc9ac58ee194b2c9e4301a2fca7b8f92508a,13718492,https://www.semanticscholar.org/paper/29a6fc9a...,Density Peaks Clustering with Differential Pri...,Density peaks clustering (DPC) is a well-known...,Conference on Innovative Data Systems Research,2017.0,23,3,0,...,NaN,NaN,conf/cidr/GuoM17,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421,e5fb0f5badaad5c04617247f000cfd4025127db0,259365527,https://www.semanticscholar.org/paper/e5fb0f5b...,Research on the profit mechanism of data eleme...,"In today’s era of big data, the data itself gr...",2023 IEEE 3rd International Conference on Info...,2023.0,7,0,0,...,1114-1118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
492,479018820d4ddeab58297224f0462497668f57bd,260956674,https://www.semanticscholar.org/paper/47901882...,Multi-label topic classification model of COVI...,One of the most significant aims of natural la...,6th International Conference on Advanced Elect...,2023.0,13,0,0,...,127870W - 127870W-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594,7733f0c3571cbc8b2ffa13086d21dc3798b505b9,271404420,https://www.semanticscholar.org/paper/7733f0c3...,PatchFinder: A Two-Phase Approach to Security ...,Open-source software (OSS) vulnerabilities are...,International Symposium on Software Testing an...,2024.0,47,1,0,...,NaN,NaN,conf/issta/0002Z0LLC24,NaN,2407.17065,NaN,NaN,NaN,NaN,NaN


# Pre-processing

In [151]:

conf_info=df[df['publicationTypes'].apply(lambda x: 'Conference' in x)]
distinct_conferences = conf_info[['publicationVenue.id','publicationVenue.name']].drop_duplicates()

journal_info=df[df['publicationTypes'].apply(lambda x: 'JournalArticle' in x and 'Conference' not in x)]
distinct_journals = journal_info[['publicationVenue.id','publicationVenue.name']].drop_duplicates()
distinct_volumes=journal_info[journal_info['publicationVenue'].apply(lambda x: x is not None)]
distinct_volumes = distinct_volumes[['year','journal.volume','publicationVenue.id']].drop_duplicates()
print(len(distinct_conferences))
print(len(conf_info))
print(len(distinct_journals))
print(len(distinct_volumes))

520
2620
2668
8406


In [145]:
len(conf_info[conf_info['year'].isnull()])

2

In [146]:
new=distinct_conferences[distinct_conferences['publicationVenue.name'].str.contains('workshop', case=False, na=False)]
len(new)

15

In [ ]:
volume_data.head()

,volumeId,number,year
0,2024-None,NaN,2024.0
1,2021-11,11,2021.0
2,2020-87,87,2020.0
3,2025-None,NaN,2025.0
4,2019-20,20,2019.0


In [ ]:
volume_data=pd.read_csv('/Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/volume.csv')
volume_data.head()
distinct_volumes.head()

distinct_volumes=distinct_volumes.rename(columns={'journal.volume': 'number'})
merged_df = pd.merge(distinct_volumes, volume_data, on=['year', 'number'], how='outer', indicator=True)
missing_in_df2 = merged_df[merged_df['_merge'] == 'left_only']


print("\nRows in df1 but not in df2:")
missing_in_df2[['year', 'number']]


Rows in df1 but not in df2:


,year,number
0,1953.0,5
20,1973.0,44 1
63,1983.0,15
93,1988.0,91 2
98,1989.0,6 3
...,...,...
3478,2024.0,89 7
3496,2024.0,abs/2404.05300
3559,2025.0,187
3567,2025.0,220


In [ ]:
df.head()

,paperId,corpusId,url,title,abstract,venue,year,referenceCount,citationCount,influentialCitationCount,...,journal.volume,journal.pages,externalIds.MAG,publicationVenue,externalIds.DBLP,tldr,publicationVenue.alternate_issns,externalIds.ArXiv,externalIds.ACL,journal
0,debc04b9dc6befa708228618197282195d923e87,275105141,https://www.semanticscholar.org/paper/debc04b9...,Bioinspired Blockchain Framework for Secure an...,WSNs are significant components of modern IoT ...,Computers,2024.0,29,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c5447f3c64ec971b610d663f70d9e0051e3fc8d2,231668203,https://www.semanticscholar.org/paper/c5447f3c...,Catechin and curcumin interact with S protein ...,None,Scientific Reports,2021.0,43,118,8,...,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,51a47b0fcb4185e57d52c7bd0bec0aa62ae4c26c,229694061,https://www.semanticscholar.org/paper/51a47b0f...,l-Arabinose Induces the Formation of Viable No...,Vibrios spp. among other bacteria form transie...,Applied and Environmental Microbiology,2020.0,60,6,1,...,87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,994ff246c4251813382be5c27df5544a8bf59ff8,275295450,https://www.semanticscholar.org/paper/994ff246...,c-Myc-targeted therapy in breast cancer: A rev...,None,Gene,2025.0,82,1,0,...,NaN,\n 149209\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,504efb13c5a675352c3ca19c646a52bfb0baabc1,203927391,https://www.semanticscholar.org/paper/504efb13...,Functional Genomics of the Retina to Elucidate...,The retina is the light sensitive part of the ...,International Journal of Molecular Sciences,2019.0,364,9,0,...,20,NaN,2977413411,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
auth_info=df[['authors']]
auth_info.head()
auth_info['authorIds'] = auth_info['authors'].apply(lambda x: [author['authorId'] for author in x])

# Exploding the 'authorIds' column into separate rows
auth_info = auth_info.explode('authorIds', ignore_index=True)
auth_info.head()
distinct_authors = auth_info[['authorIds']].drop_duplicates()
print(len(distinct_authors))

77616


/var/folders/0r/x2rt66gj2h75vp5rrm7772br0000gn/T/ipykernel_5088/2659236334.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  auth_info['authorIds'] = auth_info['authors'].apply(lambda x: [author['authorId'] for author in x])


In [ ]:
#paper_details - extract paper node, citation edge, author and corresponding author edge,
    #volume, edition, journal, event nodes, relevant edges
    #for edition combine data with json and proceeding_location.csv + add conf_id and year
#author_details - extract author node
#paper_reviewers - reviewed_by edge
#paper_keywords - extract keyword node, keyword edge to paper
#published_in edge should contain the pages data
#is_part_of edge
#held_in edge
#create journal_volume node?

In [ ]:
import argparse
import a2_create_nodes
import a2_create_edges
import a3_create_additional_nodes

In [ ]:
data_path = f"/Users/elnararb/Documents/UPC/Semantic Data Management/data/"
output_path = f"/Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/"
args = argparse.Namespace(data_path=data_path, output_path=output_path)
a2_create_nodes.main(args)

Loaded 15199 elements from /Users/elnararb/Documents/UPC/Semantic Data Management/data/combined_papers_data.json
Loaded 15199 elements from /Users/elnararb/Documents/UPC/Semantic Data Management/data/paper_keywords.json
Added 15199 papers to /Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/paper.csv with author count 94733
Added 6683 keywords to /Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/keyword.csv from total 6683
Added 969 editions to /Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/edition.csv
Added 519 conferences to /Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/event.csv
Added 7711 volumes to /Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/volume.csv
Added 2667 journals to /Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/journal.csv
Added 77614 authors to /Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/author.cs

In [ ]:
data_path = f"/Users/elnararb/Documents/UPC/Semantic Data Management/data/"
output_path = f"/Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/"
args = argparse.Namespace(data_path=data_path, output_path=output_path)
a2_create_edges.main(args)
# there are some authors with null ids
# 26587 cited_in - bcz not all papers are present in our db

Loaded 15199 elements from /Users/elnararb/Documents/UPC/Semantic Data Management/data/combined_papers_data.json
Loaded 14224 elements from /Users/elnararb/Documents/UPC/Semantic Data Management/data/paper_reviewers.json
Loaded 15199 elements from /Users/elnararb/Documents/UPC/Semantic Data Management/data/paper_keywords.json
Added 30947 paper_isRelatedTo relations to /Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/paper_isRelatedTo_keyword.csv
Added 94403 author-wrote-paper relations from total 94733 to /Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/author_wrote_paper.csv
Added 15166 paper_corresponded_by relations to /Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/paper_correspondedBy_author.csv
Added 11537 paper_publishedIn relations to /Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/paper_publishedIn_volume.csv for journals
Added 1475 paper_publishedIn relations to /Users/elnararb/Docum

In [ ]:
data_path = f"/Users/elnararb/Documents/UPC/Semantic Data Management/data/"
output_path = f"/Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/"
args = argparse.Namespace(data_path=data_path, output_path=output_path)
a3_create_additional_nodes.main(args)
#468 for affiliated_with since authors_details have authors that are not in author.csv

Loaded 111180 elements from /Users/elnararb/Documents/UPC/Semantic Data Management/data/authors_details.json
Loaded 14224 elements from /Users/elnararb/Documents/UPC/Semantic Data Management/data/paper_reviewers_metadata.json
Added 952 to /Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/affiliation.csv
Added 1144 to /Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/author_affiliatedWith_affiliation.csv
Added 35633 to /Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/review.csv
Added 35633 to /Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/review_relations.csv


In [5]:
import json
with open('/Users/elnararb/Documents/UPC/Semantic Data Management/data/combined_papers_data.json', 'r') as f:
    all_papers = json.load(f)

for paper in all_papers:
    if paper is None:  # Skip None entries
        continue
    if ('JournalArticle' in paper.get('publicationTypes', [])
    and 'Conference' not in paper.get('publicationTypes', [])and paper.get('publicationVenue') is not None):
        venue = paper.get('publicationVenue', [])
        if isinstance(venue, dict):
            conf_id = venue.get('id')
            conf_name = venue.get('name')
        else:
            print(f"Unexpected venue format: {venue}")

In [6]:
import csv

def check_for_duplicates_within_file(csv_file):
    seen_rows = set()  # This will hold unique rows as tuples
    duplicates = []    # This will store rows that are duplicates

    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        header = next(reader)  # Skip the header if you have one

        for row in reader:
            row_tuple = tuple(row)  # Convert row to a tuple to make it hashable
            if row_tuple in seen_rows:
                duplicates.append(row)  # Add duplicate row to the list
            else:
                seen_rows.add(row_tuple)  # Add unique row to the set

    if duplicates:
        print(f"Found {len(duplicates)} duplicates within the file:")
        for dup in duplicates:
            print(dup)
    else:
        print(f"{csv_file}: No duplicates found within the file.")


In [7]:
import os
import csv
import pandas as pd
for filename in os.listdir("/Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/"):
    if filename.endswith('.csv'):
        file_path = os.path.join("/Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/", filename)
        check_for_duplicates_within_file(file_path)
        with open(file_path, 'r') as f:
            df=pd.read_csv(f)
            print(f"{filename} with columns {df.columns.tolist()}")

/Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/paper_correspondedBy_author.csv: No duplicates found within the file.
paper_correspondedBy_author.csv with columns ['paperId', 'authorId']
/Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/ed_conf_merged.csv: No duplicates found within the file.
ed_conf_merged.csv with columns ['paperId', 'venue_id', 'year', 'edition', 'location', 'pages', 'new_conference_name', 'editionId']
/Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/edition.csv: No duplicates found within the file.
edition.csv with columns ['editionId', 'edition', 'location', 'year']
/Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/author.csv: No duplicates found within the file.
author.csv with columns ['authorId', 'name']
/Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/vol_journal_map.csv: No duplicates found within the file.
vol_journal_map.csv with columns ['journ

In [8]:
import pandas as pd
df=pd.read_csv("/Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/journal_hasVolume_volume.csv")

In [9]:
df.head()

,journalId,volumeId
0,f99f77b7-b1b6-44d3-984a-f288e9884b9b,70af0ca8-7a10-4268-8f67-61569fa88cf0
1,643a31e6-3016-4604-beca-0edfc0e2a834,fac3ccb9-0c02-4a3d-ba63-8040ad0e3965
2,8506a01a-40b8-4e6f-bbb8-ce2492139c15,380797bb-bbc0-4a33-8a37-631365c91c48
3,4c1bc8cf-5a6c-4b67-8d4a-dbc2ce7a06ae,34c21898-1d67-48e7-8e97-098e303be5c9
4,91b19bbf-b6f0-46c2-bbec-6bcfb5463d4d,4158eac0-efce-410a-9fe4-2af8a08c2a7d


In [10]:
import pandas as pd
import os

for filename in os.listdir("/Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/"):
    if filename.endswith('.csv'):
        file_path = os.path.join("/Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/", filename)

        with open(file_path, 'r') as f:
            df=pd.read_csv(f)
            empty_cells = df.isna() | (df == '')  # Combines NaN and empty strings

            # Summarize empty cells per column
            print(f"{f}: Empty cells per column:")
            print(empty_cells.sum())
            rows_with_empty = df[empty_cells.any(axis=1)]
            print("\nRows with empty cells:")
            print(rows_with_empty)


<_io.TextIOWrapper name='/Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/paper_correspondedBy_author.csv' mode='r' encoding='UTF-8'>: Empty cells per column:
paperId     0
authorId    0
dtype: int64

Rows with empty cells:
Empty DataFrame
Columns: [paperId, authorId]
Index: []
<_io.TextIOWrapper name='/Users/elnararb/Documents/UPC/Semantic Data Management/data/nodes_edges/ed_conf_merged.csv' mode='r' encoding='UTF-8'>: Empty cells per column:
paperId                  0
venue_id                 0
year                     0
edition                  0
location                 0
pages                  239
new_conference_name      0
editionId                0
dtype: int64

Rows with empty cells:
                                       paperId  \
1     ebe53532339772db4d79d053f7be8e385a4cd58b   
4     29a6fc9ac58ee194b2c9e4301a2fca7b8f92508a   
11    7733f0c3571cbc8b2ffa13086d21dc3798b505b9   
27    67ecaf5395891a64ee58e505416ed1fa42cb4cc3   
31    e6e0a7adde3bf95e57db

# Deal with files that have null entries

### for creating keys I used this advice: https://medium.com/neo4j/graph-data-modeling-keys-a5a5334a1297
#### Question: do I want to allow null values for editions and volumes since there are several volumes per year - delete null volume numbers
JournalArticle junction node - if we had separate edge between paper and journal we would have to store year in paper node which is not semantically good (volumes and editions should be associated with time). Updating overhead, but the queries would be faster (Like impact factor) and we have clear understanding what volume is associated with what journal

check merge specifics in https://www.youtube.com/watch?v=oRtVdXvtD3o&list=LL&index=5&t=2307s at around min 40

You're absolutely right to raise this point. Let's examine this issue more closely:

### Denormalization in Graph Databases

In traditional relational database design, we often strive for normalization to reduce data redundancy. However, graph databases operate under different paradigms, and some level of denormalization can be beneficial or even necessary.

#### Pros of this approach:
1. **Performance**: Direct relationships allow for faster traversals.
2. **Clarity**: The structure clearly represents the real-world relationship between journals and volumes.
3. **Flexibility**: It's easier to add journal-specific volume metadata if needed.

#### Cons:
1. **Data Redundancy**: Volume numbers and years are repeated across journals.
2. **Update Complexity**: If volume information needs to be changed, it must be updated in multiple places.

### Alternative Normalized Approach

A more normalized approach could look like this:

```cypher
(:Journal)-[:HAS_VOLUME]->(:VolumeInstance)-[:INSTANCE_OF]->(:VolumeDefinition {number: 1, year: 2021})
```

This structure separates the concept of a volume (VolumeDefinition) from its instance in a specific journal (VolumeInstance).

#### Pros:
1. **Reduced Redundancy**: Volume number and year are stored only once.
2. **Easier Updates**: Changing volume information affects all instances automatically.

#### Cons:
1. **Complexity**: Queries become more complex, potentially impacting performance.
2. **Less Intuitive**: The model is less straightforward and may be harder to work with.

### Recommendation

While the normalized approach reduces redundancy, the denormalized approach (separate Volume nodes for each journal) often aligns better with graph database best practices:

1. **Graph Thinking**: It represents the domain more naturally.
2. **Performance**: It optimizes for read operations, which are typically more frequent in academic publication scenarios.
3. **Scalability**: It allows for easier addition of journal-specific volume metadata.

### Balancing Act

The choice between normalization and denormalization in graph databases often comes down to use case specifics:

- If volume data rarely changes and query performance is crucial, the denormalized approach is preferable.
- If data consistency and ease of updates are more important, and you can afford slightly more complex queries, the normalized approach might be better.

In practice, a hybrid approach might be optimal, where some data is denormalized for performance, while other aspects maintain a more normalized structure.

Remember, in graph databases, relationships are first-class citizens, and some redundancy is often accepted to optimize for traversal performance and query simplicity.

---
Answer from Perplexity: pplx.ai/share